In [1]:
import sys
import ROOT as RT
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, DelphesSchema
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import pprint
import hist

Welcome to JupyROOT 6.24/06


Helper function for streamlining the import of the root files

In [2]:
def getEvents(fileURL, treename):
    return NanoEventsFactory.from_root(fileURL, treepath = treename, schemaclass = DelphesSchema).events()

Import root files and save them to dict called datasets

In [3]:
filedir   = "root://cmsxrootd.fnal.gov//store/user/aaportel/hadd/"
filenames = {'1e1mm': "m1_ct10mm.root",
             '1e2mm': "m1_ct100mm.root",
             '1e3mm': "m1_ct1000mm.root",
             '1e4mm': "m1_ct10000mm.root"}
treename  = "Delphes"

# TODO: ak.zip this into one awkward array
datasets = {}
for key in filenames:
    datasets[key] = getEvents(filedir + filenames[key], treename)


In [4]:
#help(h1.plot_pull('normal'))

A precut that takes in event.llp or event.CscCluster130 and cuts everything outside the bounding box.

In [5]:
#takes in either event.llp or event.CscCluster130
def inCSC(llp, zmin, zmax):
    
    r = np.sqrt(llp.X**2 + llp.Y**2) 
    
    trueIn = ((abs(llp.Eta) < 2.4) &
               (abs(llp.Z) > zmin) & (abs(llp.Z) < zmax) &
               (r < 6955)) 
    
    return llp[trueIn]

Generates histograms for efficiency mapping of llp and CscCluster130 for a given variable, inside of the detector

In [6]:
# initialize some dicitonaries to store histograms in an organized manner
pos_histos = {}
Ehad_histos  = {}

# for variable binning
zmin, zmax = 6500, 11000
Ehad_bins = np.array([0,25,50,75,100,150,200,250])*1.
znumbins = 30
Enumbins = 7

#zmin, zmax = 6500, 11000
#Ehad_bins = np.array([0,2500])*1.
#znumbins = 1
#Enumbins = 1

# initialize dictionaries
for ctau in datasets:
    pos_histos[ctau] = {'reco': RT.TH1F('Z decay pos (mm)', ctau + ' reco', znumbins, zmin, zmax),
                        'real': RT.TH1F('Z decay pos (mm)', ctau + ' real', znumbins, zmin, zmax)}
    
    Ehad_histos[ctau]  = {'reco': RT.TH1F('Hadronic Energy (GeV)', ctau + ' reco', Enumbins, Ehad_bins),
                          'real': RT.TH1F('Hadronic Energy (GeV)', ctau + ' real', Enumbins, Ehad_bins)}   
                         # real means generated llp (events.llp)
                         # reco means reconstructed (events.CscCluster130)


for ctau in pos_histos:
    for kind in pos_histos[ctau]:
        pos_histos[ctau][kind].Sumw2()
        pos_histos[ctau][kind].SetDirectory(0)
        Ehad_histos[ctau][kind].Sumw2()
        Ehad_histos[ctau][kind].SetDirectory(0)
        
pprint.pprint(pos_histos)

{'1e1mm': {'real': <cppyy.gbl.TH1F object at 0x556c2e3e80b0>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3ff6e0>},
 '1e2mm': {'real': <cppyy.gbl.TH1F object at 0x556c2e3f09c0>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3f04a0>},
 '1e3mm': {'real': <cppyy.gbl.TH1F object at 0x556c2e3f4700>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3e96a0>},
 '1e4mm': {'real': <cppyy.gbl.TH1F object at 0x556c2e3e3ec0>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3e1470>}}


Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Z decay pos (mm) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Hadronic Energy (GeV) (Potential memory leak).
Warning in <TROOT::Append>: Replaci

In [7]:
# loops over each ctau in datasets
for ctau in datasets:
    #sys.stdout.write("\r Reading " + filenames[ctau])
    
    # cut's out everything not in muon cap
    realLLP = inCSC(datasets[ctau].llp, zmin, zmax)
    recoLLP = inCSC(datasets[ctau].CscCluster130, zmin, zmax)
    
    # flattens to have an iterable list of all the Z positions (within the endcap)
    realLLP_zpos = ak.flatten(realLLP.Z, axis = None)
    recoLLP_zpos = ak.flatten(recoLLP.Z, axis = None)
    
    # flattens to have an iterable list of all the hadronic energies (of particles within the endcap)
    realLLP_Ehad = ak.flatten(realLLP.Ehad, axis = None)
    recoLLP_Ehad = ak.flatten(recoLLP.Ehad, axis = None)
    
    # histogram maker. Split into different loops for readability
    for z in realLLP_zpos:
        pos_histos[ctau]['real'].Fill(abs(z))
        
    for z in recoLLP_zpos:
        pos_histos[ctau]['reco'].Fill(abs(z))
        
    for Ehad in realLLP_Ehad:
        Ehad_histos[ctau]['real'].Fill(Ehad)
        
    for Ehad in recoLLP_Ehad:
        Ehad_histos[ctau]['reco'].Fill(Ehad)
        
    #print('ratio of real items ' + ctau + ':' + str(len(realLLP_zpos)/len(realLLP_Ehad)))
    #print('ratio of reco items ' + ctau + ':' + str(len(recoLLP_zpos)/len(recoLLP_Ehad)))

    print(ctau + 'Zpos reco num: ' + str(len(recoLLP_zpos)))
    print(ctau + 'Ehad reco num: ' + str(len(recoLLP_Ehad)))
    print(ctau + 'Zpos real den: ' + str(len(realLLP_zpos)))
    print(ctau + 'Ehad real den: ' + str(len(realLLP_Ehad)))

    
    print('recoreal inlcusive zpos ' + ctau + ':' + str(len(recoLLP_zpos)/len(realLLP_zpos)))
    print('recoreal inclusive Ehad ' + ctau + ':' + str(len(recoLLP_Ehad)/len(realLLP_Ehad)))
    



1e1mmZpos reco num: 3082
1e1mmEhad reco num: 3082
1e1mmZpos real den: 9570
1e1mmEhad real den: 9570
recoreal inlcusive zpos 1e1mm:0.32204806687565307
recoreal inclusive Ehad 1e1mm:0.32204806687565307
1e2mmZpos reco num: 3961
1e2mmEhad reco num: 3961
1e2mmZpos real den: 12850
1e2mmEhad real den: 12850
recoreal inlcusive zpos 1e2mm:0.3082490272373541
recoreal inclusive Ehad 1e2mm:0.3082490272373541
1e3mmZpos reco num: 2694
1e3mmEhad reco num: 2694
1e3mmZpos real den: 12200
1e3mmEhad real den: 12200
recoreal inlcusive zpos 1e3mm:0.22081967213114753
recoreal inclusive Ehad 1e3mm:0.22081967213114753
1e4mmZpos reco num: 1807
1e4mmEhad reco num: 1807
1e4mmZpos real den: 12800
1e4mmEhad real den: 12800
recoreal inlcusive zpos 1e4mm:0.141171875
recoreal inclusive Ehad 1e4mm:0.141171875


In [8]:
for ctau in datasets:
    realLLP = inCSC(datasets[ctau].llp, zmin, zmax)
    realLLP_zpos = ak.flatten(realLLP.Z, axis = None)
    realLLP_Ehad = ak.flatten(realLLP.Ehad, axis = None)

    recoLLP = inCSC(datasets[ctau].CscCluster130, zmin, zmax)
    recoLLP_zpos = ak.flatten(recoLLP.Z, axis = None)
    recoLLP_Ehad = ak.flatten(recoLLP.Ehad, axis = None)



In [9]:
RT.gStyle.SetErrorX(0)
colorDict = {'1e1mm':RT.kRed,'1e2mm':RT.kBlue,'1e3mm':RT.kOrange,'1e4mm':RT.kGreen+3}
shapeDict = {'1e1mm':RT.kFullCircle,'1e2mm':RT.kFullSquare,'1e3mm':RT.kFullTriangleUp,'1e4mm':RT.kFullTriangleDown}

# calculates and adds efficiencies to the histogram dictionaries
for ctau in datasets:
    pos_histos[ctau]['ratio'] = RT.TEfficiency(pos_histos[ctau]['reco'], pos_histos[ctau]['real'])
    pos_histos[ctau]['ratio'].SetMarkerStyle(21)
    pos_histos[ctau]['ratio'].SetLineColor(colorDict[ctau])
    pos_histos[ctau]['ratio'].SetMarkerColor(colorDict[ctau])
    pos_histos[ctau]['ratio'].SetMarkerStyle(shapeDict[ctau])
    
    Ehad_histos[ctau]['ratio'] = RT.TEfficiency(Ehad_histos[ctau]['reco'], Ehad_histos[ctau]['real'])
    Ehad_histos[ctau]['ratio'].SetMarkerStyle(21)
    Ehad_histos[ctau]['ratio'].SetLineColor(colorDict[ctau])
    Ehad_histos[ctau]['ratio'].SetMarkerColor(colorDict[ctau])
    Ehad_histos[ctau]['ratio'].SetMarkerStyle(shapeDict[ctau])
    
pprint.pprint(pos_histos)

{'1e1mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x556c2a10dd40>,
           'real': <cppyy.gbl.TH1F object at 0x556c2e3e80b0>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3ff6e0>},
 '1e2mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x556c2ead24a0>,
           'real': <cppyy.gbl.TH1F object at 0x556c2e3f09c0>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3f04a0>},
 '1e3mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x556c2e465350>,
           'real': <cppyy.gbl.TH1F object at 0x556c2e3f4700>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3e96a0>},
 '1e4mm': {'ratio': <cppyy.gbl.TEfficiency object at 0x556c2ead31f0>,
           'real': <cppyy.gbl.TH1F object at 0x556c2e3e3ec0>,
           'reco': <cppyy.gbl.TH1F object at 0x556c2e3e1470>}}


Saves efficiency histograms to pdf with proper labels

In [10]:
EhadAllFileName = 'EhadEffAll.pdf'
c = RT.TCanvas()
c.Print(EhadAllFileName + "[")
l = RT.TLegend(.85,.70,1,1)

count = 0
for ctau in Ehad_histos:
    te = Ehad_histos[ctau]['ratio']
    te.SetTitle("Hadronic Energy Efficiency Map; Hadronic Energy [GeV]; \epsilon")
    l.AddEntry(te, ctau)
    if count > 0:
        te.Draw('0 same')
    else:
        te.Draw()
        count +=1
l.Draw('same')
c.Print(EhadAllFileName)
c.Clear()
    
c.Print(EhadAllFileName + "]") #closes pdf
c.Close() #closes canvas

EhadFileName = 'EhadEff.pdf'
c = RT.TCanvas()
c.Print(EhadFileName + "[")
    
for ctau in Ehad_histos:
    te = Ehad_histos[ctau]['ratio']
    te.SetTitle(ctau+"; Hadronic Energy [GeV]; \epsilon")
    te.Draw()
    c.Print(EhadFileName)
    c.Clear()
    
c.Print(EhadFileName + "]") #closes pdf
c.Close() #closes canvas

Info in <TCanvas::Print>: pdf file EhadEffAll.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEffAll.pdf
Info in <TCanvas::Print>: pdf file EhadEffAll.pdf has been closed
Info in <TCanvas::Print>: pdf file EhadEff.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file EhadEff.pdf
Info in <TCanvas::Print>: pdf file EhadEff.pdf has been closed


In [11]:
posAllFileName = 'posEffAll.pdf'
c = RT.TCanvas()
c.Print(posAllFileName + "[")
l = RT.TLegend(.85,.70,1,1)

count = 0
for ctau in pos_histos:
    te = pos_histos[ctau]['ratio']
    te.SetTitle("Geometric Efficiency Map; Z decay pos [mm]; \epsilon")
    l.AddEntry(te, ctau)
    if count > 0:
        te.Draw('same')
    else:
        te.Draw()
        count +=1
l.Draw('same')
c.Print(posAllFileName)
c.Clear()
    
c.Print(posAllFileName + "]") #closes pdf
c.Close() #closes canvas

posFileName = 'posEff.pdf'
c = RT.TCanvas()
c.Print(posFileName + "[")
    
for ctau in pos_histos:
    te = pos_histos[ctau]['ratio']
    te.SetTitle(ctau+"; Z decay pos [mm]; \epsilon")
    te.Draw()
    c.Print(posFileName)
    c.Clear()
    
c.Print(posFileName + "]") #closes pdf
c.Close() #closes canvas

Info in <TCanvas::Print>: pdf file posEffAll.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file posEffAll.pdf
Info in <TCanvas::Print>: pdf file posEffAll.pdf has been closed
Info in <TCanvas::Print>: pdf file posEff.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file posEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file posEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file posEff.pdf
Info in <TCanvas::Print>: Current canvas added to pdf file posEff.pdf
Info in <TCanvas::Print>: pdf file posEff.pdf has been closed
